In [105]:
import pandas as pd 
#read the file 
data = pd.read_csv("data/dataset-1.csv") 
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [106]:
#Replace true and false by 1 and 0 respectively
data['churn'].replace(True, 1, inplace=True)
data['churn'].replace(False, 0, inplace=True)

In [107]:
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0.0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0.0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0.0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0.0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0.0


In [108]:
#Discretization by binning
data['account length'] = pd.cut(data['account length'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['number vmail messages'] = pd.cut(data['number vmail messages'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day minutes'] = pd.cut(data['total day minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day calls'] = pd.cut(data['total day calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total day charge'] = pd.cut(data['total day charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve minutes'] = pd.cut(data['total eve minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve calls'] = pd.cut(data['total eve calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total eve charge'] = pd.cut(data['total eve charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night minutes'] = pd.cut(data['total night minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night calls'] = pd.cut(data['total night calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total night charge'] = pd.cut(data['total night charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl minutes'] = pd.cut(data['total intl minutes'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl calls'] = pd.cut(data['total intl calls'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])
data['total intl charge'] = pd.cut(data['total intl charge'],10,right=True,labels=[0,1,2,3,4,5,6,7,8,9])

In [109]:
data['international plan'].replace('no', 0, inplace=True)
data['international plan'].replace('yes', 1, inplace=True)
data['voice mail plan'].replace('no', 0, inplace=True)
data['voice mail plan'].replace('yes', 1, inplace=True)

In [110]:
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,5,415,382-4657,0,1,4,7,6,7,...,5,5,5,4,5,4,1,4,1,0.0
1,OH,4,415,371-7191,0,1,5,4,7,4,...,6,5,6,4,6,6,1,6,1,0.0
2,NJ,5,415,358-1921,0,0,0,6,6,6,...,6,3,3,4,3,6,2,6,0,0.0
3,OH,3,408,375-9999,1,0,0,8,4,8,...,5,1,4,3,4,3,3,3,2,0.0
4,OK,3,415,330-6626,1,0,0,4,6,4,...,7,4,4,6,4,5,1,5,3,0.0


In [111]:
data.groupby('churn').count()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
churn,,,,,,,,,,,,,,,,,,,,
0.0,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850,2850
1.0,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483,483


In [112]:
data = data.drop(columns=['phone number','area code','state'])

In [113]:
from scipy.spatial import distance
import numpy as np

# Compute the Manhattan Distance Matrix
matrix = distance.cdist(data.iloc[:,1:],data.iloc[:,1:], 'cityblock')
distanceMatrix = pd.DataFrame(matrix)
distanceMatrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3323,3324,3325,3326,3327,3328,3329,3330,3331,3332
0,0.0,15.0,22.0,26.0,24.0,18.0,19.0,24.0,19.0,18.0,...,28.0,17.0,16.0,28.0,22.0,18.0,20.0,25.0,26.0,13.0
1,15.0,0.0,23.0,41.0,21.0,29.0,28.0,27.0,26.0,21.0,...,21.0,14.0,23.0,21.0,21.0,13.0,31.0,24.0,37.0,16.0
2,22.0,23.0,0.0,24.0,18.0,18.0,33.0,20.0,27.0,30.0,...,28.0,15.0,16.0,22.0,24.0,32.0,20.0,19.0,20.0,21.0
3,26.0,41.0,24.0,0.0,28.0,20.0,33.0,16.0,33.0,38.0,...,40.0,27.0,20.0,36.0,32.0,34.0,22.0,33.0,22.0,37.0
4,24.0,21.0,18.0,28.0,0.0,20.0,31.0,20.0,25.0,34.0,...,22.0,9.0,14.0,22.0,14.0,24.0,14.0,21.0,20.0,29.0


In [114]:
list(data)

['account length',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls',
 'churn']

In [115]:
data.head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,5,0,1,4,7,6,7,5,5,5,5,4,5,4,1,4,1,0.0
1,4,0,1,5,4,7,4,5,6,5,6,4,6,6,1,6,1,0.0
2,5,0,0,0,6,6,6,3,6,3,3,4,3,6,2,6,0,0.0
3,3,1,0,0,8,4,8,1,5,1,4,3,4,3,3,3,2,0.0
4,3,1,0,0,4,6,4,4,7,4,4,6,4,5,1,5,3,0.0


In [116]:
data.loc[:,'distance'] = distanceMatrix.sum(axis=1)

In [117]:
# Sort the dataset in ascending order of distance
data.sort_values(by=['distance']).head()

,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn,distance
1964,6,0,0,0,5,6,5,5,5,5,4,4,4,5,2,5,1,0.0,53716.0
262,4,0,0,0,4,6,4,5,6,5,4,4,4,5,1,5,1,0.0,53846.0
2285,1,0,0,0,4,6,4,5,5,5,5,4,5,5,1,5,1,0.0,54367.0
825,3,0,0,0,4,6,4,5,5,5,4,5,4,5,2,5,1,0.0,54693.0
1926,1,0,0,0,4,6,4,5,6,5,4,4,4,4,1,4,2,0.0,54771.0


In [155]:
# Gaussian Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
clf = GaussianNB()
results = []
for zoneLength in range(100,1001,100):
    LDT = data[0:zoneLength]
    UDT = data[-zoneLength:]
    trainingData = data[zoneLength:-(zoneLength)]
    X = trainingData.loc[:,['account length',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls']]
    Y = trainingData.loc[:,['churn']]
    clf.fit(X,Y)
    prediction = clf.predict(LDT_x)


    LDT_accuracy = accuracy_score(LDT_y, prediction) * 100
    LDT_precision = precision_score(LDT_y, prediction) * 100
    LDT_recall = recall_score(LDT_y, prediction) * 100
    LDT_fmeasure = f1_score(LDT_y, prediction) * 100
    
    UDT_x = UDT.loc[:,['account length',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls']]
    UDT_y = UDT.loc[:,['churn']]
    
    prediction = clf.predict(UDT_x)


    UDT_accuracy = accuracy_score(UDT_y, prediction) *100
    UDT_precision = precision_score(UDT_y, prediction) * 100
    UDT_recall = recall_score(UDT_y, prediction) * 100
    UDT_fmeasure = f1_score(UDT_y, prediction)  * 100
    
    results.append({'zoneLength':zoneLength, 'LDT_accuracy':LDT_accuracy, 'LDT_precision':LDT_precision, 'LDT_recall':LDT_recall, 'LDT_fmeasure': LDT_fmeasure,'UDT_accuracy':UDT_accuracy, 'UDT_precision':UDT_precision, 'UDT_recall':UDT_recall, 'UDT_fmeasure': UDT_fmeasure})
    

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [156]:
resultsDf = pd.DataFrame(results)
resultsDf = resultsDf[['zoneLength', 'LDT_accuracy','UDT_accuracy','LDT_precision','UDT_precision','LDT_recall','UDT_recall','LDT_fmeasure','UDT_fmeasure']]
resultsDf

,zoneLength,LDT_accuracy,UDT_accuracy,LDT_precision,UDT_precision,LDT_recall,UDT_recall,LDT_fmeasure,UDT_fmeasure
0,100,77.0,89.000000,25.000000,65.000000,17.647059,76.470588,20.689655,70.270270
1,200,77.0,86.500000,25.000000,55.555556,17.647059,64.516129,20.689655,59.701493
2,300,78.0,86.666667,30.769231,54.347826,23.529412,56.818182,26.666667,55.555556
3,400,78.0,85.250000,30.769231,54.687500,23.529412,53.846154,26.666667,54.263566
4,500,78.0,85.400000,30.769231,53.246753,23.529412,52.564103,26.666667,52.903226
5,600,77.0,85.666667,25.000000,55.056180,17.647059,51.578947,20.689655,53.260870
6,700,79.0,86.142857,35.714286,57.657658,29.411765,56.140351,32.258065,56.888889
7,800,79.0,86.000000,35.714286,58.730159,29.411765,55.223881,32.258065,56.923077
8,900,80.0,87.333333,38.461538,60.902256,29.411765,56.643357,33.333333,58.695652
9,1000,81.0,86.500000,42.857143,61.486486,35.294118,53.846154,38.709677,57.413249
